In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ast
from scipy.stats import kurtosis




## Loading

In [2]:
"""
purpose: parse raw data/label file, return data as numpy array
"""
def parse_file(dataFileName):
    f=open(dataFileName)
    strL=f.read()
    l=ast.literal_eval(strL)
    return np.array(l)


  

## Filtering
### eyeblinks, poor connection, band pass filter

In [3]:
"""
purpose: find the indeces of trails that possibly contaminated by eyeblinks
"""
def eyeblink_identifier(brainwave_vec, window_interval,num_of_trials, moving_step, rejection_std):
    kurtosis_vec = np.array([])
    idx =0
    while (idx < brainwave_vec.size-window_interval):
        np.append(kurtosis, kurtosis(brainwave_vec[idx:idx+window_interval]))
        idx += moving_step
    kurtosis_thres = np.mean(kurtosis_vec)+ rejection_std*np.std(kurtosis_vec)
    bad_kurtosis_index = kurtosis_vec > kurtosis_thres
    
    num_windows_per_trial = (brainwave_vec.size-window_interval)/ moving_step+1
    bad_trial_index = []
    
    for i in range(0,kurtosis_vec.size):
        if (bad_kurtosis_index[i] == True):
            bad_train_index.append(np.floor(i/(1.0*num_windows_pertrial)))
    
    return np.unique(bad_trial_index)

"""
purpose: find the indices of trials that have bad connection
"""

def bad_connection_identifier(stimuli_per_trial,num_of_trials, connection_quality_vec,rejection_rate):
    bad_trial_index =[]
    for i in range(0, num_of_trials-1):
        bad_sum = 0
        for j in range(i*stimuli_per_trial,(i+1)*stimuli_per_trial):
            if (connection_quality_vec[j] > tres):
                bad_sum += 1
        if (bad_sum/(1.0)*stimuli_per_trial > rejection_rate):
            bad_trial_index.append(i)
            
    return np.array(bad_trial_index)
            
    
     
    


In [4]:
a = [2]
a.append(2)
np.unique(a)

NameError: name 'np' is not defined

## Processing
### Chop the data into trials, discard bad trials, average trails with the same stimuli

In [5]:
f="signal_data\signal03_11_2019__10_57_39.txt" #signal data
g="stimuli_data\stimuli03_11_2019__10_57_39.txt" #stimuli stamp

#signalFile format: (outputlevel, timestamp, quality)
#stimuliFile format: (,,timestamp)
#output format:
def sync(signalFile,stimuliFile,scopePre,scopePost):   
    f=parse_file(signalFile)
    g=parse_file(stimuliFile)
    temp=[]
    out=[]
    for i in g:
        temp=[]
        for j in f:
            if j[1]>i[2]+scopePre and j[1]<i[2]+scopePost:
                temp.append(j)
        out.append(temp)
    return out

ff=sync(f,g,-0.2,1.3)

def shiftMeanTo0(data):
    m=np.mean(data,0)[0]
    return [i -m for i in data]

def getSignal(data):
    return [i[0] for i in data]

def synAll(signalFile,stimuliFile):
    return

#Test
res=shiftMeanTo0(ff[0])
# plt.plot(getSignal(res))
sig=getSignal(res)
alphaBand=butter_bandpass_filter(np.array(sig), alphaCuts[0], alphaCuts[1], fs)
plt.plot(alphaBand)



IOError: [Errno 2] No such file or directory: 'signal_data\\signal03_11_2019__10_57_39.txt'

## Classification